In [ ]:
!git clone https://github.com/l1905kw/nlp-class-project.git

fatal: destination path 'nlp-class-project' already exists and is not an empty directory.


In [ ]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
!pip install captum

In [ ]:
import spacy

import torch
import torchtext
import torchtext.data
import torch.nn as nn
import torch.nn.functional as F

from torchtext.vocab import Vocab

from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

nlp = spacy.load('en')


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        embedded = self.embedding(text)
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [ ]:
TEXT = torchtext.data.Field(lower=True, tokenize=str.split, use_vocab=True, batch_first=True)
# E vs. I
#LABEL = torchtext.data.Field(is_target=True, preprocessing=lambda x: 0 if x[0]=='I' else 1)
# 16 classes
LABEL = torchtext.data.Field(use_vocab=True, is_target=True, batch_first=True)

In [ ]:
from torchtext.data import TabularDataset

train_data, valid_data, test_data = TabularDataset.splits(path='nlp-class-project/preprocess_new',
                                   #train='personal_data_aug_train.tsv',
                                   train='original_train/mbti.tsv',
                                   #validation='split_val/mbti.tsv',
                                   #train='original_train/mbti.tsv',
                                   validation='original_val/mbti.tsv',
                                   #test='personal_data_aug_test.tsv',
                                   test='original_test/mbti.tsv',
                                   format='tsv',
                                   fields=[('label', LABEL), ('text', TEXT)])

In [ ]:
TEXT.build_vocab(train_data)

# 16 classes
LABEL.build_vocab(train_data)
output_dim = len(LABEL.vocab)

#print(len(LABEL.vocab))


In [ ]:
model = CNN(vocab_size = len(TEXT.vocab), 
            embedding_dim = 256,
            n_filters = 100,
            filter_sizes = [1,2,3,4,5],
            output_dim = output_dim,
            dropout=0.1,
            pad_idx = TEXT.vocab.stoi['<pad>'])
model.cuda()

CNN(
  (embedding): Embedding(122976, 256, padding_idx=1)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(1, 256), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(2, 256), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(3, 256), stride=(1, 1))
    (3): Conv2d(1, 100, kernel_size=(4, 256), stride=(1, 1))
    (4): Conv2d(1, 100, kernel_size=(5, 256), stride=(1, 1))
  )
  (fc): Linear(in_features=500, out_features=18, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
model.load_state_dict(torch.load('drive/My Drive/mbti_process_new_person_cnn_model_7.pt'))
model.eval()

CNN(
  (embedding): Embedding(122976, 256, padding_idx=1)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(1, 256), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(2, 256), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(3, 256), stride=(1, 1))
    (3): Conv2d(1, 100, kernel_size=(4, 256), stride=(1, 1))
    (4): Conv2d(1, 100, kernel_size=(5, 256), stride=(1, 1))
  )
  (fc): Linear(in_features=500, out_features=18, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))
lig = LayerIntegratedGradients(model, model.embedding)
token_reference = TokenReferenceBase(reference_token_idx=TEXT.vocab.stoi['<pad>'])

In [ ]:
# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 64, label = 0):
    #text = [tok.text for tok in nlp.tokenizer(sentence)]
    text = [tok for tok in str.split(sentence)]
    if len(text) < min_len:
        text += ['<pad>'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = max(min_len, len(text))

    # predict
    preds = forward_with_sigmoid(input_indices)
    pred, pred_ind = torch.max(preds, 1)
    pred = pred.data.tolist()[0]
    pred_ind = pred_ind.data.tolist()[0]
    #print(pred_ind)
    #print(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, target=label, \
                                           n_steps=50, return_convergence_delta=True)

    #print('pred: ', LABEL.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    #add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    return attributions_ig
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            LABEL.vocab.itos[pred_ind],
                            LABEL.vocab.itos[label],
                            LABEL.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [ ]:
data = sorted(test_data, key=lambda x : len(x.text))[:50]
for d in data[:10]:
  interpret_sentence(model, ' '.join(d.text), label=LABEL.vocab.stoi[d.label[0]])

In [ ]:
visualization.visualize_text(vis_data_records_ig)

In [ ]:
from tqdm.notebook import tqdm

vis_data_records_ig = []
mbti_word_importance = {}
mbti_word_occurence = {}
for i in LABEL.vocab.stoi:
  mbti_word_importance[i] = {}
  mbti_word_occurence[i] = {}

for data in tqdm(valid_data):
  attr = interpret_sentence(model, ' '.join(data.text), label=LABEL.vocab.stoi[data.label[0]])

  attr_sum = torch.sum(attr, dim=2).squeeze().tolist()
  for i, w in enumerate(data.text):
    mbti_word_importance[data.label[0]][w] = mbti_word_importance[data.label[0]].get(w, 0) + attr_sum[i]
    mbti_word_occurence[data.label[0]][w] = mbti_word_occurence[data.label[0]].get(w, 0) + 1

In [ ]:

import pickle
with open('drive/My Drive/mbti_aug_analysis_result.pt', 'wb') as f:
    pickle.dump({'word_importance': mbti_word_importance, 'word_occurence': mbti_word_occurence}, f, pickle.HIGHEST_PROTOCOL)

'''
with open('drive/My Drive/mbti_analysis_result.pt', 'rb') as f:
    data = pickle.load(f)
'''

In [ ]:
import pandas as pd

df = pd.DataFrame()
for t, value in sorted(mbti_word_importance.items()):
  if t == '<unk>' or t== '<pad>':
    continue
  w = list(value.items())
  sorted_words = sorted(w, key=lambda s: s[1], reverse=True)[:50]

  if(len(sorted_words)) < 50:
    continue

  df[t] = [w[0] for w in sorted_words]
  #df[t + '_importance'] = [w[1] for w in sorted_words]
  #df[t + '_occurence'] = [mbti_word_importance[t][w[0]] for w in sorted_words]
  #df[t + '_norm_importance'] = [w[1] / mbti_word_occurence[t][w[0]] for w in sorted_words]

display(df)
df.to_csv("word_importance.csv")
  #print(t, sorted_words)
  #most_importance_words = sorted([w for w, importance in importances.item()])

,ENFJ,ENFP,ENTJ,ENTP,ESFJ,ESFP,ESTJ,ESTP,INFJ,INFP,INTJ,INTP,ISFJ,ISFP,ISTJ,ISTP
0,enfj,enfp,entj,entp,intps,esfp,estj,estp,infj,infp,intj,intp,isfj,isfp,istj,istp
1,!,!,entjs,entps,isfj,!,istj,estps,infjs,infps,intjs,intps,fe,!,istjs,istps
2,enfjs,enfps,an,the,welcome,?,an,fun,feel,feel,.,),an,feel,happy,.
3,feel,an,intj,an,esfp,:d,enfp,an,!,really,),would,feel,really,you,","
4,an,feel,mbti,you,intp,typed,istp,istp,my,feeling,?,the,!,),posting,'ll
5,infj,i,.,?,'m,intj,not,shit,-,and,not,is,would,music,mbti,really
6,all,fun,not,fun,from,her,all,got,you,!,by,",",like,'m,the,shit
7,really,:d,shit,sep,as,enfp,!,car,se,",",are,sep,entj,'ll,rant,maybe
8,time,love,sep,shit,maybe,'m,infp,type,life,-,'s,use,her,istp,being,got
9,their,lol,be,'m,!,entp,by,:),fe,music,would,all,?,feelings,back,fuck


In [ ]:
norm_word_importance = {}
norm_df = pd.DataFrame()
for i in LABEL.vocab.stoi:
  norm_word_importance[i] = {}
for t, value in mbti_word_importance.items():
  if t == '<unk>' or t== '<pad>':
    continue
  for w, importance in value.items():
    norm_word_importance[t][w] = importance / mbti_word_occurence[t][w]
  
  w = list(norm_word_importance[t].items())
  sorted_words = sorted(w, key=lambda s: s[1], reverse=True)[:50]
  norm_df[t] = [w[0] for w in sorted_words]
  #norm_df[t + '_importance'] = [w[1] for w in sorted_words]
display(norm_df)
norm_df.to_csv("norm_word_importance.csv")

In [ ]:
import math

def consine_similarity(word_importance_1, word_importance_2):
  denominator_1 = denominator_2 = nominator = 0
  word_set = set(list(word_importance_1.keys()) + list(word_importance_2.keys()))
  for w in word_set:
    nominator += word_importance_1.get(w, 0) * word_importance_2.get(w, 0)
    denominator_1 += word_importance_1.get(w, 0) * word_importance_1.get(w, 0)
    denominator_2 += word_importance_2.get(w, 0) * word_importance_2.get(w, 0)
  denominator = (math.sqrt(denominator_1) * math.sqrt(denominator_2))
  if denominator == 0:
    return 0
  return nominator / denominator

print(consine_similarity({'a': 0, 'b': 1, 'c': 1, 'd': 1}, {'a': 1, 'b': 0, 'c': 1, 'd': 1})) #0.67
print(consine_similarity({'a': 1.0, 'b': -1.0}, {'a': 1.0, 'b': 0.5})) #0.3162
print(consine_similarity({'a': 1.0, 'b': -1.0}, {'a': 1.0, 'b': -1.0})) #1.0

In [ ]:
similarities = []
top_sim = []
types = []
for t, value in sorted(mbti_word_importance.items()):
  if t == '<unk>' or t== '<pad>':
    continue
  types.append(t)
  similarity = []
  for t2, value2 in sorted(mbti_word_importance.items()):
    if t2 == '<unk>' or t2== '<pad>':
      continue
    s = consine_similarity(value, value2)
    similarity.append()
  similarities.append(similarity)

sim_df = pd.DataFrame(similarities, columns=types, index=types)
display(sim_df)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(16, 16))
sns.heatmap(sim_df, linewidths=0.1, vmax=0.4,  cmap="YlGnBu",
            linecolor='white', annot=True)
plt.show()

In [ ]:
sim_for_common_letters = {}
num_common_letters = {}

for i, sims in enumerate(similarities):
  for j, sim in enumerate(sims):
    num_common_char = len(set(types[i]).intersection(types[j]))
    sim_for_common_letters[num_common_char] = sim_for_common_letters.get(num_common_char, 0) + sim
    num_common_letters[num_common_char] = num_common_letters.get(num_common_char, 0) + 1

print('Common letters', 'Avg. Similarity', sep='\t')
for common_letter, sum_sim in sorted(sim_for_common_letters.items(), reverse=True):
  if common_letter == 4:
    continue
  print('{}\t{}'.format(common_letter, sum_sim / num_common_letters[common_letter]))

In [ ]:
similarities = []
types = []
for t, value in sorted(norm_word_importance.items()):
  if t == '<unk>' or t== '<pad>':
    continue
  types.append(t)
  similarity = []
  for t2, value2 in sorted(norm_word_importance.items()):
    if t2 == '<unk>' or t2== '<pad>':
      continue
    
    similarity.append(consine_similarity(value, value2))
  similarities.append(similarity)

sim_df = pd.DataFrame(similarities, columns=types, index=types)
display(sim_df)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(16, 16))
sns.heatmap(sim_df, linewidths=0.1, vmax=0.1,  cmap="YlGnBu",
            linecolor='white', annot=True)
plt.show()

In [ ]:
sim_for_common_letters = {}
num_common_letters = {}

for i, sims in enumerate(similarities):
  for j, sim in enumerate(sims):
    num_common_char = len(set(types[i]).intersection(types[j]))
    sim_for_common_letters[num_common_char] = sim_for_common_letters.get(num_common_char, 0) + sim
    num_common_letters[num_common_char] = num_common_letters.get(num_common_char, 0) + 1

print('Common letters', 'Avg. Similarity', sep='\t')
for common_letter, sum_sim in sorted(sim_for_common_letters.items(), reverse=True):
  if common_letter == 4:
    continue
  print('{}\t{}'.format(common_letter, sum_sim / num_common_letters[common_letter]))